In [42]:
import os
import datetime

import wrds
import pandas as pd
from pandas import DataFrame
import numpy as np
from scipy.stats.mstats import winsorize

from Constants import Constants as const

In [40]:
conn = wrds.Connection(wrds_username='aheitz')

Loading library list...
Done


In [7]:
# set CRSP date range a bit wider to guarantee collecting all information
crsp_begdate = '01/01/2006'
crsp_enddate = '12/31/2017'

# get items from fundq
fundq = conn.raw_sql(f"""
                        select gvkey, fyearq, fqtr, cshoq, prccq, mkvaltq, dlcq, dlttq, ibq, 
                        saleq, atq, fyr, datafqtr
                        from comp.fundq 
                        where consol='C' and popsrc='D' and datafmt='STD'
                        and datadate between '{crsp_begdate}' and '{crsp_enddate}' 
                        """, date_cols = ['datafqtr',])

D:\Users\wangy\anaconda3\Lib\site-packages\wrds\sql.py:579: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in df:
D:\Users\wangy\anaconda3\Lib\site-packages\wrds\sql.py:579: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in df:


In [25]:
fundq['mkvaltq'] = fundq['mkvaltq'].fillna(fundq['prccq'] * fundq['cshoq'])
fundq['LOSS'] = fundq['ibq'].apply(lambda x: int(x < 0))
fundq['ROA'] = fundq['ibq'] / fundq['atq']
fundq['LEV'] = (fundq['dlcq'].fillna(0) + fundq['dlttq'].fillna(0)) / fundq['atq']
fundq['SIZE'] = fundq['mkvaltq'].apply(np.log)

fundq.replace([np.inf, -np.inf], np.nan, inplace=True)
fundq.describe()

C:\Users\wangy\AppData\Local\Temp\ipykernel_32112\793552016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundq['mkvaltq'] = fundq['mkvaltq'].fillna(fundq['prccq'] * fundq['cshoq'])
C:\Users\wangy\AppData\Local\Temp\ipykernel_32112\793552016.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundq['LOSS'] = fundq['ibq'].apply(lambda x: int(x < 0))
C:\Users\wangy\AppData\Local\Temp\ipykernel_32112\793552016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,fyearq,fqtr,cshoq,prccq,mkvaltq,dlcq,dlttq,ibq,saleq,atq,fyr,datafqtr,LOSS,ROA,LEV,SIZE
count,542916.000000,542916.000000,4.986760e+05,428593.000000,416315.000000,390801.000000,4.072670e+05,423506.000000,422404.000000,4.104750e+05,542916.000000,542767,542916.000000,406683.000000,408678.000000,416305.000000
mean,2011.589402,2.486246,3.542347e+02,28.338839,3502.792398,885.112031,2.223568e+03,48.493119,774.694171,1.130053e+04,10.439280,2011-12-17 17:11:54.585079808,0.343733,-1.187436,1.418528,5.240766
min,2005.000000,1.000000,0.000000e+00,0.000001,0.000000,-154.000000,-1.210000e-01,-61659.000000,-25623.000000,-1.267300e+01,1.000000,2005-07-01 00:00:00,0.000000,-127324.000000,-30.000000,-12.144413
25%,2009.000000,1.000000,9.760000e+00,1.600000,31.909100,0.000000,0.000000e+00,-1.376000,2.277000,3.373850e+01,10.000000,2009-01-01 00:00:00,0.000000,-0.030201,0.006680,3.462995
50%,2012.000000,2.000000,3.093700e+01,9.900000,186.758800,1.521000,1.237200e+01,0.449000,30.711000,3.164800e+02,12.000000,2012-01-01 00:00:00,0.000000,0.001837,0.164608,5.229921
75%,2015.000000,3.000000,8.312275e+01,25.820000,1143.581900,28.103000,3.937805e+02,12.458000,239.231000,1.980834e+03,12.000000,2015-01-01 00:00:00,1.000000,0.013834,0.381783,7.041966
max,2018.000000,4.000000,2.597497e+07,141600.000000,859967.798700,605462.506000,3.296298e+06,127140.000000,207307.334000,3.879172e+06,12.000000,2018-04-01 00:00:00,1.000000,22154.000000,18116.000000,13.664650
std,3.448615,1.116453,7.366978e+04,700.603629,16834.462437,12286.945124,3.518926e+04,519.492761,3951.112163,9.986418e+04,3.014712,NaN,0.474954,226.712047,44.630105,2.601151


In [26]:
columns_to_winsorize = ['ROA', 'LEV', 'SIZE']

fundq: DataFrame = fundq.drop_duplicates([const.GVKEY, 'fyearq', 'fqtr'], keep='last')

# Winsorize each column in the dataset reg_df3 at the 1st and 99th percentiles
for column in columns_to_winsorize:
    non_na_data = fundq[column].dropna()
    winsorized_data = winsorize(non_na_data, limits=[0.01, 0.01])
    fundq.loc[non_na_data.index, column] = winsorized_data


ValueError: Must have equal len keys and value when setting with an iterable

In [67]:
# Merge all the other variables with the regression data
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20240918_stock_act_reg_data_quarterly.dta'))

ac_df: DataFrame = pd.read_csv(os.path.join(const.TEMP_PATH, '2007_2016_quarterly_analyst_coverage.csv'))
df_df: DataFrame = pd.read_csv(os.path.join(const.TEMP_PATH, '2007_2016_quarterly_dispersion_fcsterror.csv'),
                               usecols=['ticker', 'quarter', 'numest', 'DISPERSION', 'FCSTERROR']).rename(
    columns={'quarter': 'fqtr'})

In [11]:
fundq.dropna(subset=['fyearq', 'fqtr'], how='any', inplace=True)
fundq['qtr'] = fundq.apply(lambda x: "%dQ%d" % (x['fyearq'], x['fqtr']), axis=1)

In [22]:
ctrl_vars = fundq[['gvkey', 'qtr', 'SIZE', 'ROA', 'LEV', 'LOSS']].rename(columns={'qtr': 'fqtr'}).dropna(how='any')
reg_df.rename(columns={'yearquarter': 'fqtr'}, inplace=True)
reg_df['fqtr'] = reg_df['fqtr'].str.replace('q', 'Q')
ctrl_vars['gvkey'] = pd.to_numeric(ctrl_vars['gvkey'], errors='coerce')
reg_df['gvkey'] = pd.to_numeric(reg_df['gvkey'], errors='coerce')
reg_df_ctrl = reg_df.merge(ctrl_vars, on=[const.GVKEY, 'fqtr'], how='inner')


In [27]:
# Winsorize each column in the dataset reg_df3 at the 1st and 99th percentiles
for column in ['ROA', 'LEV', 'SIZE']:
    non_na_data = reg_df_ctrl[column].dropna()
    winsorized_data = winsorize(non_na_data, limits=[0.01, 0.01])
    reg_df_ctrl.loc[non_na_data.index, column] = winsorized_data

In [35]:
iclink = pd.read_pickle(os.path.join(const.TEMP_PATH, '20241011_iclink.pkl'))

In [43]:
# CCM data
_ccm = conn.raw_sql(""" select gvkey, lpermco as permco, lpermno as permno, 
                        linkdt, linkenddt 
                        from crsp.ccmxpf_linktable 
                        where usedflag=1 
                        and linkprim in ('P', 'C')""", date_cols=['linkdt', 'linkenddt'])

_ccm[['permco', 'permno']] = _ccm[['permco', 'permno']].astype(int)

# Fill linkenddt missing value (.E in SAS dataset) with today's date
_ccm['linkenddt'] = _ccm.linkenddt.fillna(datetime.date.today())

_sec = conn.raw_sql(""" select ibtic, gvkey from comp.security """)


# Start the sequence of left join
gvkey = pd.merge(_ccm, _sec.loc[_sec.ibtic.notna()], how='left', on=['gvkey'])

# high quality links from iclink
# score = 0 or 1
iclink_hq = iclink.loc[(iclink.score <=1)]

gvkey = pd.merge(gvkey, iclink_hq, how='left', on=['permno'])

# fill missing ticker with ibtic
gvkey.ticker = np.where(gvkey.ticker.notnull(), gvkey.ticker, gvkey.ibtic)

# Keep relevant columns and drop duplicates if there is any
gvkey = gvkey[['gvkey', 'permco', 'permno', 'linkdt', 'linkenddt','ticker']]

gvkey = gvkey.drop_duplicates()

In [45]:
gvkey['linkenddt'] = pd.to_datetime(gvkey['linkenddt'].fillna(datetime.date.today()))

# date ranges from gvkey
# min linkdt for ticker and permno combination
gvkey_mindt = gvkey.groupby(['ticker','permno']).linkdt.min().reset_index()

# max linkenddt for ticker and permno combination
gvkey_maxdt = gvkey.groupby(['ticker','permno']).linkenddt.max().reset_index()

# link date range 
gvkey_dt = pd.merge(gvkey_mindt, gvkey_maxdt, how='inner', on=['ticker','permno'])

In [58]:
gvkey['gvkey'] = pd.to_numeric(gvkey['gvkey'], errors='coerce')

# Calculate link date ranges for givken gvkey and ticker combination
gvkey_mindt1 = gvkey.groupby(['gvkey', 'ticker']).linkdt.min().reset_index().rename(columns={'linkdt':'mindate'})
gvkey_maxdt1 = gvkey.groupby(['gvkey', 'ticker']).linkenddt.max().reset_index().rename(columns={'linkenddt':'maxdate'})
gvkey_dt1 = pd.merge(gvkey_mindt1, gvkey_maxdt1, how='inner', on=['gvkey','ticker'])

# Use the date range to merge
comp = pd.merge(reg_df_ctrl, gvkey_dt1, how='left', on =['gvkey'])
comp['datadate'] = comp.apply(lambda x: datetime.datetime(x['Year'], 3 * x['Quarter'], 1), axis=1)
comp = comp.loc[(comp.ticker.notna()) & (comp.datadate<=comp.maxdate) & (comp.datadate>=comp.mindate)]

# Sort data and drop duplicates
comp = comp.sort_values(by=['gvkey','fqtr']).drop_duplicates()

In [61]:
comp.drop(['mindate', 'maxdate', 'datadate'], axis=1, inplace=True)

In [66]:
reg_ac_df: DataFrame = comp.merge(ac_df[['gvkey', 'fqtr', 'coverage']], on=['gvkey', 'fqtr'], how='left').merge(
    ac_df[['ticker', 'fqtr', 'coverage']], on=['ticker', 'fqtr'], how='left', suffixes=('', '_tic'))
reg_ac_df['coverage'] = reg_ac_df['coverage'].fillna(reg_ac_df['coverage_tic'])
reg_ac_df.drop(['coverage_tic'], axis=1, inplace=True)
reg_ac_df.describe()

,gvkey,Year,Quarter,freq,MajorGovCustomer,post,ln_freq,d2008q1,d2008q2,d2008q3,...,d2013q4,d2014q1,d2014q2,d2014q3,d2014q4,SIZE,ROA,LEV,LOSS,coverage
count,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,...,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,99961.000000,84281.000000
mean,81162.655626,2011.016426,2.507848,0.955343,0.068657,0.428827,0.260756,0.034173,0.034854,0.034503,...,0.035904,0.035354,0.036184,0.036094,0.036814,6.361375,-0.011353,0.232472,0.336471,5.414742
std,73794.134309,1.991401,1.118589,3.464302,0.252871,0.494911,0.665531,0.181675,0.183410,0.182519,...,0.186052,0.184673,0.186749,0.186525,0.188307,2.142935,0.101081,0.263339,0.472505,4.442239
min,1004.000000,2008.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.173758,-1.934874,0.000000,0.000000,1.000000
25%,12266.000000,2009.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.886853,-0.008327,0.015237,0.000000,2.083333
50%,61766.000000,2011.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,6.367715,0.007072,0.182514,0.000000,4.142857
75%,160598.000000,2013.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,7.825089,0.018769,0.360023,1.000000,7.375000
max,317264.000000,2014.000000,4.000000,75.000000,1.000000,1.000000,4.330733,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,11.382038,0.158977,4.364706,1.000000,37.208333


In [68]:
reg_df_df: DataFrame = reg_ac_df.merge(df_df, on=['ticker', 'fqtr'], how='left')

In [71]:
sue123_df: DataFrame = pd.read_pickle(os.path.join(const.TEMP_PATH, '20241012_sue_final.pkl'))
sue123_df.head()

,gvkey,ticker,permno,conm,dgap,fyearq,fqtr,datadate,fyr,rdq,...,leadrdq1,repdats,mcap,medest,act,numest,basis,sue1,sue2,sue3
23,001004,AIR,NaN,AAR CORP,0 days,2017,2.0,2017-11-30,5,2017-12-21,...,2018-03-21,NaT,1444.03182,NaN,NaN,NaN,NaN,0.001443,0.001443,NaN
11,001004,AIR,NaN,AAR CORP,0 days,2017,1.0,2017-08-31,5,2017-09-19,...,2017-12-21,NaT,1248.46932,NaN,NaN,NaN,NaN,0.000832,0.002214,NaN
47,001004,AIR,NaN,AAR CORP,0 days,2016,4.0,2017-05-31,5,2017-07-11,...,2017-09-19,NaT,1200.32876,NaN,NaN,NaN,NaN,0.003721,0.003721,NaN
35,001004,AIR,NaN,AAR CORP,0 days,2016,3.0,2017-02-28,5,2017-03-21,...,2017-07-11,NaT,1181.43208,NaN,NaN,NaN,NaN,0.004067,0.004067,NaN
22,001004,AIR,54594.0,AAR CORP,0 days,2016,2.0,2016-11-30,5,2016-12-21,...,2017-03-21,2016-12-21,1266.39681,0.347,0.35,2.0,D,0.001898,0.001847,0.000081


In [82]:
# sue123_df['gvkey'] = pd.to_numeric(sue123_df['gvkey'], errors='coerce')
# sue123_df['qtr'] = sue123_df['fqtr']
# sue123_df['fqtr'] = sue123_df.apply(lambda x: "%dQ%d" % (x['fyearq'], x['qtr']), axis=1)
reg_sue_df: DataFrame = reg_df_df.merge(sue123_df[['gvkey', 'fqtr', 'numest', 'sue1', 'sue2', 'sue3']].dropna(subset=['gvkey']), on=['gvkey', 'fqtr'], how='left', suffixes=('', '_sue')).merge(
    sue123_df[['ticker', 'fqtr', 'numest', 'sue1', 'sue2', 'sue3']].dropna(subset=['ticker']), on=['ticker', 'fqtr'], how='left', suffixes=('', '_ticker'))

key_to_drop = list()
for key in ['sue1', 'sue2', 'sue3']:
    reg_sue_df[key] = reg_sue_df[key].fillna(reg_sue_df[f"{key}_ticker"])
    key_to_drop.append(f"{key}_ticker")
    
reg_sue_df['numest_sue'] = reg_sue_df['numest_sue'].fillna(reg_sue_df["numest_ticker"])
key_to_drop.append("numest_ticker")

reg_sue_df.drop(key_to_drop, axis=1, inplace=True)
reg_sue_df.describe()

,gvkey,Year,Quarter,freq,MajorGovCustomer,post,ln_freq,d2008q1,d2008q2,d2008q3,...,LEV,LOSS,coverage,numest,DISPERSION,FCSTERROR,numest_sue,sue1,sue2,sue3
count,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,...,100353.000000,100353.000000,84673.000000,81091.000000,7.100100e+04,7.992200e+04,68703.000000,89311.000000,89371.000000,68607.000000
mean,81045.812502,2011.016691,2.508156,0.952169,0.068668,0.428776,0.259874,0.034100,0.034817,0.034448,...,0.232796,0.335625,5.419973,22.648605,3.637994e+04,1.073945e+05,6.229495,0.001883,0.002075,-0.001688
std,73772.048146,1.990824,1.118745,3.459076,0.252889,0.494904,0.664594,0.181486,0.183317,0.182379,...,0.263076,0.472211,4.440893,19.677158,2.633499e+06,7.911955e+06,5.761351,1.095427,0.610267,0.093145
min,1004.000000,2008.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,-2.155172e-01,-1.392145e+03,0.000000,-40.154839,-11.897638,-10.775758
25%,12256.000000,2009.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.015708,0.000000,2.090909,8.000000,6.497726e-04,8.190008e-04,2.000000,-0.008723,-0.007743,-0.001317
50%,61750.000000,2011.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.183382,0.000000,4.142857,17.000000,1.594310e-03,2.650214e-03,4.000000,0.000868,0.000932,0.000551
75%,160571.000000,2013.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.360392,1.000000,7.375000,32.000000,4.716327e-03,1.001025e-02,8.000000,0.008489,0.007618,0.002914
max,317264.000000,2014.000000,4.000000,75.000000,1.000000,1.000000,4.330733,1.000000,1.000000,1.000000,...,4.364706,1.000000,37.208333,149.000000,3.626189e+08,1.268182e+09,48.000000,301.807107,155.052158,5.113095


In [86]:
# Winsorize each column in the dataset reg_df3 at the 1st and 99th percentiles
for column in ['DISPERSION', 'FCSTERROR']:
    non_na_data = reg_sue_df[column].dropna()
    winsorized_data = winsorize(non_na_data, limits=[0.025, 0.025])
    reg_sue_df.loc[non_na_data.index, column] = winsorized_data
    
reg_sue_df.describe()

,gvkey,Year,Quarter,freq,MajorGovCustomer,post,ln_freq,d2008q1,d2008q2,d2008q3,...,LEV,LOSS,coverage,numest,DISPERSION,FCSTERROR,numest_sue,sue1,sue2,sue3
count,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,100353.000000,...,100353.000000,100353.000000,84673.000000,81091.000000,71001.000000,79922.000000,68703.000000,89311.000000,89371.000000,68607.000000
mean,81045.812502,2011.016691,2.508156,0.952169,0.068668,0.428776,0.259874,0.034100,0.034817,0.034448,...,0.232796,0.335625,5.419973,22.648605,0.021592,0.066881,6.229495,0.001883,0.002075,-0.001688
std,73772.048146,1.990824,1.118745,3.459076,0.252889,0.494904,0.664594,0.181486,0.183317,0.182379,...,0.263076,0.472211,4.440893,19.677158,0.079411,0.264163,5.761351,1.095427,0.610267,0.093145
min,1004.000000,2008.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,0.000034,0.000000,0.000000,-40.154839,-11.897638,-10.775758
25%,12256.000000,2009.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.015708,0.000000,2.090909,8.000000,0.000650,0.000819,2.000000,-0.008723,-0.007743,-0.001317
50%,61750.000000,2011.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.183382,0.000000,4.142857,17.000000,0.001594,0.002650,4.000000,0.000868,0.000932,0.000551
75%,160571.000000,2013.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.360392,1.000000,7.375000,32.000000,0.004716,0.010010,8.000000,0.008489,0.007618,0.002914
max,317264.000000,2014.000000,4.000000,75.000000,1.000000,1.000000,4.330733,1.000000,1.000000,1.000000,...,4.364706,1.000000,37.208333,149.000000,0.462783,1.546667,48.000000,301.807107,155.052158,5.113095


In [87]:
reg_sue_df.to_stata(os.path.join(const.RESULT_PATH, '20241012_stock_act_quarterly_regression.dta'), 
                   write_index=False, version=117)